In [1]:
import csv, io
import json
from bs4 import BeautifulSoup
import codecs, prettytable
import requests
import codecs
from selenium.webdriver import Chrome


header_data={
		"User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36",
		"Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
		"referer" : "https://forum.gamer.com.tw/B.php?bsn=60559&subbsn=7"
		}

In [2]:
# 取得所有標題的 snA

# --------subbsn----------
Apple = "7" 
p_Apple = 18

Sony = "13" 
p_Sony = 9

Samsung = "12" 
p_Samsung = 10

XiaoMi = "11" 
p_Mi = 6

Google = "22" 
p_Google = 2

# 需設定參數
classNow = Apple 
page_count = p_Apple 
def get_snA(page_count, classNow):
    snA_list = []
    for page in range(1, page_count+1):
        r1 = requests.get(
                f"https://forum.gamer.com.tw/B.php?bsn=60559&subbsn={classNow}",
                headers=header_data,
                params={
                    "page":str(page)
                    }
            )
        b1=BeautifulSoup(r1.text, "html.parser")
        a1 = b1.find_all("td",{"class":"b-list__summary"})


        for a2 in a1:
            snA = a2.find("a")
            # print(snA.attrs["name"])
            snA_list.append(snA.attrs["name"])
            
    snA_list.remove("48476")
        
    return snA_list

snA_list = get_snA(page_count, classNow)
print(snA_list)
print(len(snA_list))



['50083', '50073', '50076', '50066', '50064', '50038', '50021', '50023', '49991', '49985', '49962', '49938', '49906', '49898', '49896', '49894', '49886', '49874', '49857', '49698', '49819', '49795', '49788', '49785', '49771', '49721', '49724', '49702', '49526', '49688', '49656', '49679', '49661', '49652', '49640', '49600', '49581', '49539', '49512', '49493', '49468', '49407', '49398', '49394', '49373', '49287', '49271', '49264', '49262', '49253', '49084', '49221', '49176', '49209', '49186', '49120', '49117', '49087', '49102', '49094', '49063', '49059', '48971', '48884', '48860', '48851', '48823', '48794', '48765', '48777', '48770', '48768', '48759', '48734', '48711', '48699', '48634', '48598', '48613', '48583', '48590', '48571', '48503', '48504', '48485', '48484', '48479', '48469', '48438', '48432', '48404', '48365', '48320', '48296', '48297', '48182', '48166', '48164', '48087', '48077', '46876', '42799', '47418', '46126', '47356', '47224', '47167', '47131', '46629', '46972', '46839', 

In [9]:

# snA_list.append("49698")
# print(snA_list)
# snA_list.index("42749")
a = snA_list.index("45213")
snA_list = snA_list[a:]
# snA_list

In [4]:
def get_floor(bs):
    floor_list=[]
    floor = bs.find_all("p",{"class":"BH-pagebtnA"})
    
    for f in floor:
#         print(f.text[-1])
        floor_list.append(f.text[-1])
#   print(floor_list)
    return floor_list

In [5]:
def get_page_parser(snA, classNow):
    r = requests.get(f"https://forum.gamer.com.tw/C.php?bsn=60559&snA={snA}&subbsn={classNow}",
                     headers=header_data,
                     params={
                     }
                     )
    r.encoding = "utf-8"
    bs = BeautifulSoup(r.text, "html.parser")
    return bs

In [6]:
def get_commentId(snA, floor, classNow):
    r2 = requests.get(
                f"https://forum.gamer.com.tw/C.php?bsn=60559&snA={snA}&page={floor}&subbsn={classNow}",
                headers=header_data,
                params={
                    }
            )
#   print(range(1, int(page_list[0]) + 1))
    print("正在處理第幾頁:", floor)
    commentId_list=[]
    b2=BeautifulSoup(r2.text, "html.parser")
    a1 = b2.find_all("textarea")
    for a in a1:
#             print(a.attrs["data-snb"])
        commentId_list.append(a.attrs["data-snb"])
    print(commentId_list)
    return commentId_list


In [10]:
import time

# 取得該snA展開留言編號

for snA in snA_list:
    
    data = []
    
    bs = get_page_parser(snA, classNow)
    floor_list = get_floor(bs)

    driver = Chrome("./chromedriver")
    for floor in range(1, int(floor_list[0]) + 1):
        
        commentId_list = get_commentId(snA, floor, classNow)
        # 打開網址
        driver.get(f"https://forum.gamer.com.tw/C.php?bsn=60559&snA={snA}&page={floor}&subbsn={classNow}")
        # find -> find-element
        # find all -> find_elements
        for id_ in commentId_list:      
            try:
                time.sleep(3)
                driver.find_element_by_id("showoldCommend_"+ id_).click()
                time.sleep(3)
            except:
                continue   
#     -------------------------------------------------------------------------------------            
        comment = driver.find_elements_by_class_name("reply-content__article")
        content = driver.find_elements_by_class_name("c-article__content")
        title = driver.find_elements_by_class_name("c-post__header__title ")
        title_list = []
        content_list = []
        comment_list = []
        result = dict()

        for til in title:
            title_list.append(til.text)
            print(til.text)
        for con in content:
            content_list.append(con.text)
            print(con.text)   
        for com in comment:
            comment_list.append(com.text)
            print(com.text) 
        
        result["title"] = title_list
        result["content"] = content_list
        result["comment"] = comment_list
        data.append(result)
        
        with open("Apple_new.csv", "a", encoding="utf-8", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["標題", "內容", "使用者評論"])
            for item in data:
                writer.writerow([item['title'], item['content'], item['comment']])
                     
    driver.close() 
        

正在處理第幾頁: 1
['136068']
【問題】Apple的yt怪怪的
如題，小弟現在是用apple的6+，系統版本是12.2，每次我yt要往下滑（就是關影片）的時候，常常會超大lag怎麼按鎖定或主畫面鍵都沒用，但卻還一直有聲音，按大小聲鍵也沒用，有沒有人知道是為什麼&如何解決？
壽命到了....
信仰不足 果粉要說是背景播放
該換了，他壽命已盡
重置看看，不能就送修or買新手機
北京播放
我的往下滑也會大lag 手機是iPhone X 版本iOS 13 beta3
土豪IPX這樣應該很不正常吧，手邊5年多前的三星J和送給老妹的2年多前的IP7+都很正常。
小弟仍是學生...只能撿爸媽用過的二手機
ra1xp 我前三個YouTube更新版本都沒問題
等下次更新看看了
正在處理第幾頁: 1
['136049']
【問題】I8+螢幕莫名抖動
https://cdn.discordapp.com/attachments/569881768841314324/596903410737872917/video.mov
先看這個影片
尤其在充電的時候特別嚴重
常常誤觸
有人知道這是怎樣嗎嗚嗚
手機賣到現在大概1年3個月吧
有觸碰在螢幕上嗎？？沒有就看要不要送檢，有保護貼先撕了測試看看
保護貼有撕下來過了 還是抖
但是比較常是我長按的時候會抖
然後充電的時候短觸容易誤觸
按下面跑去上面這樣
電磁干擾.......emi的關係
手機殼也拔掉試試
有些手機殼容易產生靜電
nanri 那是什麼
此地無淫 我來試試
豪ㄘ的水餃去google emi啊，就電磁波干擾到觸控模組，才會在那邊抖抖抖，通常太爛的充電器會造成這種結果
只不過你平時沒充電也會這樣子的話，恐怕觸控模組有問題了，建議送檢比較能知道原因了
充電器一定不是原廠？
正在處理第幾頁: 1
['135943']
【問題】日版iPhone Xs 熱點
各位版上大大們好
日前友人在日本購入iPhone Xs
確認買的版本是sim-free的
插上台灣sim卡可用
但他不知道問什麼不能分享熱點
請問大大們有遇到這種情況嗎？
該如何解決呢
先在此謝謝各位
不知道 但幫推
不知道 但本人我在日本買的7＋可以分享
GR56 Kotori 看來好像沒有人知道...是不是買到機王了
正在處理第幾頁: 1
['135873']
【問題】XS M

想問能擋Google瀏覽器的廣告嗎
樓上推薦的AdGuard最近有5折優惠33台幣
不過好多設定要調  不太會用...
——
啊...剛剛看上面說是不是說變成不能擋app的廣告了
是不是白買了
我目前擋廣告是就乾脆全部社群網站都用safara的版本
巴哈姆特 fb ig 跟一些常用網站都擋的掉
只是要習慣一下
我之前有買 ADGUARD 你把 DNS 設置成他們家的基本上就擋 80% 了 再靠這個 app 找出有廣告的鏈結 然後手動加到黑名單 會用的話其實是很有效果
手機 Youtube APP 那個播影片前的廣告應該是擋不了的 我主要是用來擋各大新聞網的煩人蓋版
YAO 好謝謝你的教學與意見
改用edge 內建擋廣告的軟體 好用多了
王大人 我是想其他app的廣告也ban掉 不只是瀏覽器
我是用下面那個+ADGUARD的DNS 下面那個之前免費時候載了 光用他本體沒什麼P用
蜥蜴長老 還好沒有買，現在就只用adguard已經很好
ADGUARD連app那種彈出式的都擋 然後yt的廣告可以用別的app擋享受手機全無廣告
FutureMind的AdBlock在今年2月中被Apple下架了.
蜥蜴長老 免費時候下載+1 這個App原本真的有用 但不知道為什麼到了 2017 2018 慢慢失效 現在已經不使用它了 (後續好像還是有推出更新版 但一樣擋不了廣告) 後來改使用 AdGuard 明顯好很多
顯示VPN是因為 他就類似一條水管 在中間先幫你過濾
那麼其他apps的內置廣告大部分都能擋嗎？
例如巴哈手機app的廣告
マンシューりー 我用安卓的可以
Hooo_發飆哥 可惜啊 我用的是ios
可是iOS要鎖了
了解熊熊的初心者 熊熊甚麼意思？
就是不會上架了
封鎖其它app內廣告違反Apple App Store政策, AG Pro暫時會很長一段時間沒法更新, 目前還在架上, 只是被Apple禁止更新.
IOS麻煩地方就在這裡，他鎖了你就別想用
免費版只能用在safari 而且只局限於safari的廣告不能擋其他apps內嵌的，而付費版不止可以擋safari而且連巴哈和其他apps也都能擋
所以可以擋的
我想買了
https://adguard.com/zh_tw/welcome.html
他們官網比較解得詳細
不過youtube 和 twitch的影片廣告還是不能擋


7會比較好

二手價格差不多性能卻差很多


不過小尺寸手機續航力要注意就是了
我以前初支手機也是安卓，前後換了四支，
平均一支用不到一年就非常頓，連撥話都頓，
後來受不了，106年初換了蘋果手機I7+用到現在，

108年4月電池健康度僅剩82%，預約台北101，
換了新電池，手機又和新的一樣，

I7+是不錯用，目前玩傳說，寶可夢還很順，
但建議你，考慮要再撐幾個年，
I8+是不錯的選擇。
手機排班傷眼抱歉
i7
但i7實際上現在也是堪用而已，體驗上不會好到哪去
要不要考慮直接上i8...
A11還能跟S855打
我用I7+使用起來會覺得有些卡了，建議上I8或者XR
Ip7現在只是堪用，要就上ip8，另外你安卓現在是用？
I6就算了，記憶體小到連新版iOS要順跑都有問題
I6S是好一些，不過，二手買來之後要先煩惱電池問題
我比較好奇你是用到哪一款安卓
A12不是跟855嗎？
雞包紙包雞很有可能是中階機型吧？遇到很多買了安卓中階機的人跳apple，然後再說安卓機爛的人XD
Neco 想太多855什麼咖
Point 真的假的= = A12那麼誇張喔
我的手機充電孔壞了，現在用HTC m8 好幾年前的那隻
有玩手游會課金嗎？無課再考慮蘋果
現在最好優先挑3gb記憶體的機種
M8啊，也快五年了，這五年內手機SOC進步很大，app也越來越吃記憶體了，你M8會用到受不了，其實也不意外啦
你才宅妳全家都宅 無課
nanri 其實是那Nokia6.1p不過充電孔壞了
如果充電孔修要花2000以上就考慮直接換了o.o
Nokia 6.1P?這隻是中階偏近入門機(S636+4G/64G)啊，難怪你會用到抓狂
不過說真的...充電孔會在不到一年之內就自爆，還真的很無言，保固內送修應該免費維修吧？
A10會卡....？ 我看是軟體問題吧
果然，又是一個用中低階安卓電子垃圾想換蘋果的…
這三支選i7，i6 i6s系列停產了，之後零件可能比較不好找
雞包紙包雞 這種人挺多的XD...所以蘋果越傳越神
M8當初哪是中低階...明明是旗艦機...
五年跟不上是正常的 安卓手機平均換機是兩年
手機用久了也沒以前順，可以的話是重刷rom全部清空會好一點
X洨妹我是指nokia6.1
X洨妹他是用Nokia 6.1P用到受不了了，我原本也以為他是用M8用5年XD
我的6s+根本沒卡 不要亂說好嗎
換

小狼妳爸媽買房也便宜優惠在我們這代阿(我八年級生) 對吧 整個經濟面不會因為這種個人因素而停滯
諾雅基本上扯到經濟面有點遠了，我是不覺得漲這樣會增加什麼經濟面的程度，到是會增加通膨程度而已
小狼你汽油每公升上漲，有人拒加嗎? 有通膨嗎?
諾雅說真的現在汽油比我大學那時候便宜多了....我那時候再炒什麼全球石油危機快沒油可以開採了，漲到30多元....
小狼另外 現在上漲的叫做跨境電商稅，原本這稅收都是廠商自行吸收的，現在只是吐出來還給我們罷了
諾雅還有你拿民生用品來當例子說真的不太好，汽油上漲影響很大，因為有些地方是用發電機來發電，發電機就是用油，油漲了，他們成本也會漲，全體東西都會漲
諾雅但!!至於會不會降價!!!會!!有些東西是會降價的，這就是市場機制調節，但有些不太會降就是看商人良心，至於蘋果以後會不會把2%降回去?我們拭目以待就知道了
小狼其實早就有以水代油的架構能驅車了，只是企業體去制止了這方面的發展，因為沒油水撈了
諾雅如果最後沒降，就代表庫克贏了!!庫克又再一次成功搾取蘋果信仰的費用
諾雅沒唷，現在有多2%不是稅收的錢，而是蘋果自行加價的
小狼沒什麼，我只是覺得你只是認為你還是學生沒錢享受沒調漲的時間去課金而已
最後一句，會買的還是會買，不會買的一樣不會買，蘋果一年比一年貴，它們收入一樣很可觀，我股票一樣賺很爽
等你開始每年都在納稅後，你就知道這10%根本沒什麼了
諾雅舉學生只是例子，並不是我XD，而且很多人可能有各種因素這邊只是隨便說一種，對我來說是沒差拉，我雖然有課金但也沒課很多我幾乎沒影響
小狼人家蘋果抽這個稅，你反觀看成手續費就行了，以前都是平台商跟開發商抽取，現在吐回來給我們 自然很多東西會以新的方式來操作
諾雅只是比較反感的是，蘋果當初裝清流，現在又想全回收(多+2%)，我真的只能說沒那個屁股就不要在那邊裝清流，然後現在又多加價想全部回收
諾雅尤其是以前看到那些吹捧蘋果自行吸收多好多好的腦粉，現在看起來，滿噁心的...
小狼回你的例子跟因素問題，課金不是必需，這論點只有供需
小狼庫克不比賈伯斯，謹記
現在的果粉，幾乎都是庫克時代才入教的吧。賈伯斯時代的果粉才沒那麼多，也沒那麼無腦
安卓也不一定不會跟上吧，說的太滿真的不是好事；我看商店漲價的不少、也有自行吸收的
Re_Dash哈哈，我就是賈伯斯時期的果粉，其實改朝換代，真的看的是相當清楚..

飛刀之刃
https://www.ithome.com/0/420/780.htm
是的，我不否認現在破兆，我也很看好Aruze，用既有的微軟組合包往雲端服務走，這方向非常正確，只是就微軟市值超越蘋果這點做個補充。 事實上亞馬遜在這過程根本被忽視。
飛刀之刃他天天發廢文吹微軟踩蘋果
沙比 我只是就是論事而已 你看TESTV 的配音君也一樣不看好蘋果就從蘋果全家通變成微軟全家通了
沙比 是踩所有微軟以外的
SkyactivXR 我看你是嫉妒微軟才會這樣說吧....
SkyactivXR 還有你不要每次人生攻擊好嗎 趕快去雞雞陰德好嗎
亞馬遜：全世界老百姓都在微軟google蘋果，沒有人知道我維持市值世界前三已經好幾年了 (X
我其實那段也不是跟樓主講的，只是在說不要長期接收單一方面的資訊，而對於該狀況有不符事實的理解，進而忽視實際狀況。
剛好這篇就是這例子，講微軟市值超越蘋果講了幾個月，自然會被誤導成蘋果長期市值被微軟超越，但事實上這拉鋸戰已經打了半年了，前三名破過兆的都還在互搶第一。
少來了John大，您老以前多少偏頗、錯誤的文章被刪呀… 沉寂一陣子以為所有人都忘了嗎/_>\
SkyactivXR 那又怎樣 你是不是每次都文不對題才來有事沒事阿 你是吃飽太閒阿你
還有那偏三爽的 我都沒說到微軟 你為何以為我會說道微軟 你是我肚子裡面的蛔蟲阿
微軟好棒棒 讚
繼new iPad mini太貴，Google Pixel低規高賣，下次還有什麼，好期待ㄛ
X洨妹 pixel 本來就高價低貴好嗎 PTT 上的人很多人都在嫌的說
https://www.ptt.cc/bbs/MobileComm/M.1555317354.A.677.html
PTT看世界 哈哈，你知道Pixel售價一直都不便宜嗎? 再者全球同步更，無限雲端，這些都免錢就是了?
X洨妹 但是又不是每個人都像你一樣接受這樣價格阿
JOHN很大你有在注意Google手機的市佔嗎? 去看過再來講你在講啥
X洨妹 我知道啊 很多美國人在用 在台灣跟本很少看到.....我目前只有去書展才看到有人在用Pixel 的說
ix系列那種設計跟自己想要的差太多吧
JOHN很大 人身攻擊啦，人生攻擊也太可怕
pixel從來都不是賣cp值 是賣超長期的系統更新及更新神速 還有Google Camera
蒼_GCam繁化模式 我寧可低配

...

到底關賈伯斯什麼事情= =?CEO不見得主宰一切行銷好嗎？
反而sony,htc,asus保密到家，發表才知道外型長怎樣(x
上面都太年輕，不知道賈伯斯是誰。
懶人包：太多人了 管不了有心人嘴巴
都是暴雷仔的錯
因為沒有賈伯斯了，沒有賈伯斯的蘋果已經不是蘋果了 +1
只是間吃老本的資訊商業公司而已
太白了啦
只是間把mark貼在任何3c產品上騙粉錢的公司,你怎麼會有他們保密工作會做好的幻想呢?
買的人變少 就調高價錢30-50%優秀的公司
正在處理第幾頁: 1
['121609']
【情報】蘋果2018秋季發表會 iPhone XR、XS、Max展示機影片
看好看準下好離手
Youtube 小齒輪 可自動翻譯 繁體中文



Apple Watch Series 4
正在處理第幾頁: 1
['121578']
【問題】XS MAX 256 512讀取速度
以過往的經驗

高容量會在讀取速度上勝過低容量

但往往看到的文章都是32或64跟128以上的做比較

這次256跟512倒是第一次

不知道各位果粉如果不考慮其他因素
256跟512素上差異會很大嗎

單看容量的話，容量越大代表 Flash 顆粒數越多，I/O 數越多，代表可以同時寫入的資料量越大，就好比電影院散場時，出口的數量越多，就可以越快疏散完人潮。
以上是我爬文看到的

我是個想換APPLE的新用戶
來問大家這個菜菜的問題 希望別見怪
沒差，要用蘋果請不要在意任何有關"數字"的東西
速度差不了多少 容量差了很多
差了4倍... 可是用不到
雪空4倍是說64跟256嗎
喔沒看到你說256 我是用128跟512比
理論上有差 但幾乎感覺不出來
給予世界痛苦恩恩的確測試機器有差 但用戶無法感受到差異
蘋果的讀寫一直都是頂尖水準，所以其實根本沒差
正在處理第幾頁: 1
['121555']
【情報】2018全新iPhone發佈會懶人包！！！
熬夜把這次的Apple iPhone Xs、Xs Max 以及 iPhone XR的發布會給看完了，自己剪接了一下發布會的影片!!
並做了字幕讓大家能快速知道這次三隻手機的重點。
希望大家看了會喜歡!!!
感謝大大製作懶人包!
謝謝整理 蘋果又帶來創新跟未來
感謝，小米9外觀發佈會
懶人包就是留錢買GTX2080
偽乳酸飲料RTX
顯示卡跟手機是不同的東西吧....
正在處理

IMAC因為操作上跟微軟不一樣 所以有比較便利點
安卓陣營有一系列的商品我目前只知道小米比較多，手機和他的智能家庭設備可以做連動，其他安卓廠商在設備連動就非常少
信基督教的不太會改信佛教吧
阿貴我是覺得，除非真的資金很充足加上信仰很深，不然不太可能這樣買一堆週邊
我姐買手機 但沒買桌機 沒買筆電 沒買ipad 她是不是假果粉
Re_Dash 應該說有錢人大部分會挑的牌子就那幾樣XDD 你這樣說有點偏哈哈
自從賈柏斯掛了後，蘋果就不曾讓人有驚艷感了......至於操作簡易性還是看老人領悟力，要說卓越真的未必
小狼 要做設備連動就要弄到生態圈那樣各類產品都有(或合作)，這種除非有在認真砸錢經營不然我認為一般廠商做不來
一般只要能做到適配支援就已經很強了XD
照蘋果現在的狀態被拉下只是"時間"，畢竟這幾間中國大廠在好幾年前就已經做好新技術研發人才儲備跟專利應用項目研發的比重增加了很多
現在只是剛開花而已，後面的百花盛開就才是蘋果的真危險期
再危險也是安卓陣營先被陸商打趴，畢竟同系統；品牌忠誠度比較低
安卓系統是美國，不是中國大陸，像中興事件大陸已暴露只能夠吹牛，關鍵東西還是用美國貨
系統大家都不敢亂更新就笑笑的了
卓粉表示瀏海又不是蘋果發明的（那你們為何今年才抄？）
因為愛~~~
所以愛~~
珍惜在一起的愉快
一分開
好險三爽沒跟風...不然真的快沒手機可買了
醜的有剩，結果一大票跟進...干這是科技美的又一次死亡
最可笑的就是小米 自稱全面屏的先驅 結果反過來抄劉海 結果還是有下巴阿 這不就是為了抄而抄嗎 原本有MIX那樣的技術 結果弄出米8那個樣子出來
虛無殿有神麼好可笑的 小米的確是全面屏的先驅啊 MIX和米8是不同產品線 有不同的設計語言這不是很正常嗎 不喜歡瀏海就去買MIX2S啊
乖小孩看起來就是你沒看懂我的意思 小米明明能做出mix那種技術了 卻還要反過來抄劉海 人家蘋果全面屏確實只剩瀏海 結果你小米抄完有下巴
乖小孩那下巴無法去除的情況下 你為什麼還要特地去抄劉海? 這不就是為了抄而抄嗎 雖說大概為了和mix區分不用相同技術 但我認為米8還有比瀏海更好的方式
這劉海簡直牛逼死了
不會全部倒閉，不喜歡用ios的人多得是
你想多了 用安卓幾乎不會像改ios
5>5s 指紋解鎖 5s>6大螢幕（雖然很多安卓已經是大螢幕） 6>6S 3D Touch雖然使用率不高 

每朝健康爽健美茶 因為那是對應Spen增加新東西，什麼文字懸浮翻譯，三星就是靠這個走出安卓
如果照你這樣分 那這樣搞的人就不是Note粉了...
每個品牌都有人把新機的新功能移植刷到舊機上 我倒認為這樣刷仍然算那個品牌的粉絲...
蠻想知道oppo多出的價值在哪....
每朝健康爽健美茶 三星系統更新慢，所以我不選三星很久了，我最早接觸三星是它的平板
N照騙 有人就愛這一味
N 你自拍連修圖都省了，還有oppo會灑錢要遊戲廠商們幫它們特定幾款機型作優化。
X洨妹大概是國際品牌太深入在地化造成的 後綴F的國際版很多國家在用 但裡面的天線設計還是有點不同 所以才要一個一個去更新...
同意說法，真的信仰就不會在那邊東越獄西刷機，要的就是原汁原味。
不過我仍認為蘋果比其他手機好030
璀璨天空 沒有東西是十全十美的，不過手機很多人去選就要注意了，真的只是推銷話術廣告多嗎？這點值得思考
我的理由來自於我的經驗0.0廣告一邊去，實際體驗比較重要
小米連oppo和vivo都打不贏，規格真的是一切？ 只看規格的人真的很好笑
我覺得MIUI真的蠻好用的啦 用過這麼多訂製OS來說
那些在嘴MIUI廣告的只有三種人，一個是使用中國版MIUI的，另一個是設定都不會開的人，另一種比較厲害是根本沒用過
MIUI如果你很熟～應該都知道最後只有“個性主題”那個程式會有廣告
其他都很容易關掉
換句話說基本上只要不是白癡連設定都不會按，MIUI只有一個小地方會有廣告
小米文件管理器 就看到廣告了...
就是因為不好用才會刷機阿，呵呵
話說，蘋果售後服務是如何？
每朝健康爽健美茶一樣設定裡可以關，整個MIUI就只有個性主題的廣告關不掉
MIUI會用的只要root之後不用刷也可以很乾淨
拿U11 有點想等滿兩年換i8 plus..
怕你撐不到兩年就GG了
沒啥問題老實說 已經一年多了 也沒出現啥問題 還是一樣順
「這台車明明可以調成500匹馬力啊，為什麼只給我350匹」
「這台車明明極速可以跑到300多啊，為什麼要限制在250？」
我一直都覺得小米是提供了刷機的服務給某些用家群，也沒有說一定要刷機，只是可以體驗不同的東西而已，至少他的意思是不同人都可以用不同方法來享受
過往沒到智能手機的時代前都是這樣，一個品牌出很多款手機，當時還沒有效能這麼一回事，刷機我個人觀感是換個保護套而已
世狸你說得很對 但大多

我喜歡你打ㄉ風格
°BAHA柏宏〞★原來你是貓咪那篇XD 我想說開箱文太多了 來稍微細談一下iPhoneX的使用體驗
Android手機早就有可以完整錄製影像和聲音的功能了…
不，我不是貓咪那篇，【開箱"聞"】iPhoneX/銀-開箱就是要聞啊，不然要幹嘛https://forum.gamer.com.tw/C.php?bsn=60559&snA=33085
彥仔有嗎 我記得規定是不行的 我之前在開發的時候有想要用 才發現安卓是不開放這個API使用 還是新版有內建了 我開發是在7的版本
°BAHA柏宏〞★歐歐 原來是你 哈哈 認錯人 XD
ML和DP是啥？但是python我知道
誰找我!?我是貓咪那篇的主人XD
machine learning跟dynamic programming
外觀太爛了吧 劉海全屏，後鏡頭，真的對設計師寫個服字
愛著雛田的忍者感謝
OLED的壽命平均大約是多久?出問題時會怎麼樣?
現在移動端已經能辦到在線學習了喔 進步真大
python也可以接C，只要有用到GPU前端甚麼語言都可以
可以接C理所當然也可以接Swift.........(?)
凜嵐人講太多程式的東西了 還是刪掉好了XD 不過我也不太懂蘋果是怎麼用這個自稱神經網路引擎的 蘋果也不可能向Google一樣發paper分享...
滄海瀏海我用玩後覺得還行 但是僅限於豎屏 橫屏就會怪怪的這是真的 然後那個鏡頭直的到沒什麼感觸 看過硬體內部 鏡頭是被Sensor的配置
滄海強制被趕到角落去 所以變成了直的設計 這次iPhoneX的內部非常密 壞了維修費沒AppleCare應該會非常可觀...
QooQPOWER大概兩到三年年左右是OLED的壽命吧 OLED用久了顏色會不精準(比起LCD速度快) 尤其是藍色的會衰退很快
QooQPOWER換個方式講就是曇花一現？ 還有烙印的困擾 不過現在烙印有軟體輔助去避免 實用習慣別太差通常不會開燒XD
我也是ip5 , 還在考慮要不要換說...現在也是有些災情，看來還是先等一陣子吧，話說...IPX好像不能換新機只能維修（是謠傳嗎？）
壞熊熊 我聽說蘋果旗艦店買的話 是要去檢測 檢測完確定是機王才會換新的給你 如果是電信的話有個別不同的手法 聯強神腦處理方式都不一樣
了解，感謝解答
中肯的分享文,GP奉上
請問樓主平常是做什麼研究工作的??
Andy 

記得手機內就有音效晶片，再透過這條線轉接，裡面也有音效晶片，這樣算額外加料?不過看起來結果還算正面
也許以後會有廠商推出具有DAC晶片的轉接頭吧
外放會比HTC10好嗎？
蘋偉是比較大聲、音質輸HTC 10
蘋偉要去哪裡買啊
富堡手機行
怕
現在買有加送板手造型OTG隨身碟和手機殼
幫我訂台
我夠膽說如果有3.5，測試結果都是沒差多少
不會到時候 連轉接線都能出發燒線巴 ex:李x線.管x線.黑x石線等..XD~
為什麼是拿上一代手機跟這代手機比阿，音效晶片就不一樣了吧，也聽不出來是因為音效晶片比較好還是lightening轉接3.5mm比較好
IP7出個有3.5頭的版本就能公平了，可是他沒有，只能拿上一代比較
我想問呢 這文章有有星的文會加入精華區嗎？
晶片不同就不能說是取消3.5 所以提升音質了
而且多了一段轉接線方便性大打折扣，所以不認同官方為了提升音質而刪孔的爛說法
不過大大測評還是一樣有水準只好登錄給GP xD(話說我原本還以為走錯版了
為啥我用K3003i結果完全是i7轉接起來效果有夠爛的 , 人聲的確很突出 但是樂器的聲音變成髒髒的感覺(感覺樂器變得有點像電子琴出來的感覺)
那個轉接頭很爛 會讓好耳機變的很無力 我雖然是老古董的銅渦輪 聽了也是傻了一下 哈哈哈
感覺像G5搞一個模組化，你打他一次臉V20還不是有辦法把模組音效卡直接內建進去。
有種技術未到乾脆利用品牌號召力去改變原有習慣，成功了在用“專利”去壓死人。
USB音頻3.0規範都要發布,我不看好這個接口
好文
木耳推
文章是優質的，但前提不太對，音質的提升不能單純歸咎於取消3.5mm接頭
好文，但真心覺得要多接那一條線有夠醜0.0
葉姵文是你....
IE 800 推的了嗎 -0- 沒想過那麼高級的... 不過80連中階播放都出惡聲了....
雖然完全看不懂，但還是趕快給GP不要讓別人發現
是說樓主可以試試看6s插lightning嗎
所以...可以私給我拉拉雄貼哪邊買嗎? (被敲
其實我也一直很在意那個垃垃能 想貼home鍵
我是推測是hamee的商品 但這款那邊能買到？
拉拉熊…
轉接線記得有新聞報導不能用在6s上
官網販賣轉接線的網頁有寫有Lightning的設備都能用, 基本上就是個Lightning介面的DAC而已.
然後如果有人發一篇隨身播放器跟家用訊源的比較, 那會不會改成要聽音

In [10]:
def main():
    data = []
    
    for i in range(1, 66):
        ret = get_page_detail(i)
        # data.append(ret)
        print("[", i, "]", "=====", ret)
        with open("MIUserComment.csv", "a", encoding="utf-8", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["標題", "內容", "使用者評論"])
            for item in ret:
                writer.writerow([item['title'], item['content'], item['comments']])


    #    for i in range(1, 66):

我有試過itunes，但是根本沒有辦法傳任何東西
icloud又非常慢，傳個小影片電腦好了手機的icloud資料夾要很久之後才會顯示
請問有什麼好方法嗎？
推薦一個軟體 sendanywhere，我家開照相館客人傳送照片都用這個方式，而且不限於照片，影片音訊文件都可以
手機跟電腦都安裝究OK了
既然都用蘋果了，你也只能麻煩一點的慢慢傳了，要直接把手機當隨身碟使用，無解。
感謝大家的建議，我每個都試試看好了
正解:購買一隻S10
蔬菜下一隻手機是有想換回安卓了
很棒的app
